In [ ]:
import pandas as pd
import json
import numpy as np

In [ ]:
data = json.load(open("raw_comment_responses.json"))

In [ ]:
comment_data = data['comments']

In [ ]:
comment_data[0]

In [ ]:
videoIds = [list(comment.keys())[0] for comment in comment_data]

In [ ]:
# Pass in a video (JSON response from YouTube API) return a dataframe
def getCommentsFromVideo(video):
    video_id = list(video.keys())[0]
    comments = list(video.values())[0]
    if 'noComments' in comments.keys():
        return
        # return pd.DataFrame({
        #     'videoId': video_id,
        #     'textDisplay': "",
        #     'textOriginal': "",
        #     'authorDisplayName': "",
        #     'authorProfileImageUrl': "",
        #     'authorChannelUrl':"",
        #     'authorChannelId':{'value':""},
        #     'canRate':False,
        #     'viewerRating':"",
        #     'likeCount':0,
        #     'publishedAt':"",
        #     'updatedAt':"",
        #     'commentId':""
        # })
    else:
        comments = comments['items']

    if len(comments) == 0:
        return
        # return pd.DataFrame({
        #     'videoId': video_id,
        #     'textDisplay': "",
        #     'textOriginal': "",
        #     'authorDisplayName': "",
        #     'authorProfileImageUrl': "",
        #     'authorChannelUrl':"",
        #     'authorChannelId':{'value':""},
        #     'canRate':False,
        #     'viewerRating':"",
        #     'likeCount':0,
        #     'publishedAt':"",
        #     'updatedAt':"",
        #     'commentId':""
        # })
    
    comment_data = []
    for comment in comments:
        datum = comment['snippet']['topLevelComment']
        comment_id = datum['id']
        comment_info = datum['snippet']
        comment_info['commentId'] = comment_id
        comment_data.append(comment_info)
    
    return pd.DataFrame(comment_data)

In [ ]:
# Pass in a list of videos (JSONs) and return a concatenated dataframe. Pass in 1000 video (100000 comments)
def loadVideos(data):
    dataframes = []
    for video in data:
        dataframes.append(getCommentsFromVideo(video))
    
    return pd.concat(dataframes, ignore_index=True)

In [ ]:
commentDf = pd.DataFrame()

In [ ]:
currentlyDownloaded = len(comment_data)
currentlyTransformed = len(commentDf)
currentlyDownloaded, currentlyTransformed

In [ ]:
first_1000 = loadVideos(comment_data[:1000])

In [ ]:
second_1000 = loadVideos(comment_data[1000:2000])

In [ ]:
third_1000 = loadVideos(comment_data[2000:3000])

In [ ]:
fourth_1000 = loadVideos(comment_data[3000:4000])

In [ ]:
fifth_1000 = loadVideos(comment_data[4000:5000])

In [ ]:
sixth_1000 = loadVideos(comment_data[5000:6000])

In [ ]:
dataHalf = pd.concat([first_1000,second_1000,third_1000,fourth_1000,fifth_1000,sixth_1000])

remainder = loadVideos(comment_data[6000:])

dataFull = pd.concat[[first_1000,second_1000,third_1000,fourth_1000,fifth_1000,sixth_1000,remainder]]

In [ ]:
commentsGrouped = pd.DataFrame(dataHalf[['videoId', 'textOriginal']].groupby('videoId')['textOriginal'].apply(list)).reset_index()

In [ ]:
# commentsGrouped['_id'] = mongoId
# commentsGrouped = commentsGrouped.rename(columns={"_id":"_id", "textOriginal":"comments", "videoId":"id"}).reindex(columns = ['_id', 'comments', 'id'])
commentsGrouped = commentsGrouped.rename(columns={"textOriginal":"comments", "videoId":"id"}).reindex(columns = ['comments', 'id'])

In [ ]:
commentsGrouped.iloc[2]

In [ ]:
commentsJson = commentsGrouped.to_json(orient = 'records')
with open("../pseudoscience-paper-data/groundtruth_videos_comments.json", "w") as f:
    f.write(commentsJson)